In [4]:
import pandas as pd
import numpy as np
import os

# Load the one-minute stock prices CSV file (adjust the path as needed)
file_name = "stock_prices_15m"
df = pd.read_csv(f'{file_name}.csv')

# If a Date column exists, parse it as datetime and sort by time
if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values('Date', inplace=True)
    df.reset_index(drop=True, inplace=True)
    

#############################################
# 1. MACD Calculation
#############################################
# MACD is computed using two EMAs of the closing price.
# Typical periods: 12 and 26 for EMAs and 9 for the signal line.
df['EMA12'] = df['Close'].ewm(span=12, adjust=False).mean()
df['EMA26'] = df['Close'].ewm(span=26, adjust=False).mean()
df['MACD'] = df['EMA12'] - df['EMA26']
df['Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
df['Hist'] = df['MACD'] - df['Signal']

#############################################
# 2. RSI Calculation (Period = 14)
#############################################
delta = df['Close'].diff()
gain = delta.copy()
loss = delta.copy()
gain[gain < 0] = 0
loss[loss > 0] = 0
loss = loss.abs()

window_length = 14
avg_gain = gain.rolling(window=window_length, min_periods=window_length).mean()
avg_loss = loss.rolling(window=window_length, min_periods=window_length).mean()
rs = avg_gain / avg_loss
df['RSI'] = 100 - (100 / (1 + rs))

#############################################
# 3. KDJ Calculation (Period = 9)
#############################################
# Compute the Raw Stochastic Value (RSV)
period = 9
df['Low_Min'] = df['Low'].rolling(window=period, min_periods=period).min()
df['High_Max'] = df['High'].rolling(window=period, min_periods=period).max()
df['RSV'] = (df['Close'] - df['Low_Min']) / (df['High_Max'] - df['Low_Min']) * 100

# Initialize K, D, and J. Typically start K and D at 50.
df['K'] = np.nan
df['D'] = np.nan
df['J'] = np.nan

for i in range(len(df)):
    if np.isnan(df.loc[i, 'RSV']):
        continue
    if i == df['RSV'].first_valid_index():
        df.loc[i, 'K'] = 50
        df.loc[i, 'D'] = 50
    else:
        df.loc[i, 'K'] = (2/3) * df.loc[i-1, 'K'] + (1/3) * df.loc[i, 'RSV']
        df.loc[i, 'D'] = (2/3) * df.loc[i-1, 'D'] + (1/3) * df.loc[i, 'K']
    df.loc[i, 'J'] = 3 * df.loc[i, 'K'] - 2 * df.loc[i, 'D']

#############################################
# 4. OSC Calculation: Price Oscillator
#############################################
# Using the percentage difference between 5-minute and 10-minute SMAs.
sma_fast = df['Close'].rolling(window=5).mean()
sma_slow = df['Close'].rolling(window=10).mean()
df['OSC'] = (sma_fast - sma_slow) / sma_slow * 100

#############################################
# 5. BOLL Calculation: Bollinger Bands
#############################################
# Bollinger Bands using a 20-period SMA and 2 standard deviations.
boll_period = 20
df['BOLL_Mid'] = df['Close'].rolling(window=boll_period).mean()
df['BOLL_STD'] = df['Close'].rolling(window=boll_period).std()
df['BOLL_Upper'] = df['BOLL_Mid'] + 2 * df['BOLL_STD']
df['BOLL_Lower'] = df['BOLL_Mid'] - 2 * df['BOLL_STD']

#############################################
# 6. BIAS Calculation: Price Bias Indicator
#############################################
# BIAS is the percentage difference between the closing price and its 6-period SMA.
bias_period = 6
sma_bias = df['Close'].rolling(window=bias_period).mean()
df['BIAS'] = (df['Close'] - sma_bias) / sma_bias * 100

#############################################
# Display or save results
#############################################
# Print the last few rows to check the calculated indicators
print(df.tail())

file_path = f'./preprocessed_data/{file_name}_with_indicators.csv'

# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(file_path), exist_ok=True)

# Save the DataFrame to the specified path
df.to_csv(file_path, index=False)



                       Datetime        Open        High         Low  \
7373  2025-03-14 16:45:00+00:00  165.130005  165.345001  165.070007   
7374  2025-03-14 16:45:00+00:00  606.229980  606.640015  606.000000   
7375  2025-03-14 16:45:00+00:00  387.045013  387.549286  386.910004   
7376  2025-03-14 16:45:00+00:00  121.035004  121.129997  120.809998   
7377  2025-03-14 16:45:00+00:00  248.587204  248.727600  247.929993   

           Close   Volume Ticker       EMA12       EMA26       MACD  ...  \
7373  165.345001   179078  GOOGL  242.731984  259.863374 -17.131390  ...   
7374  606.609985    72257   META  298.713215  285.548308  13.164907  ...   
7375  387.549286    95106   MSFT  312.380303  293.103936  19.276367  ...   
7376  121.129997  2081392   NVDA  282.957179  280.365126   2.592053  ...   
7377  248.359207   628401   TSLA  277.634414  277.994317  -0.359903  ...   

            RSV          K          D          J        OSC    BOLL_Mid  \
7373   9.238873  18.448442  25.282456   4